# Spørring som både gir tekst(navn) og kode(ID) fra JSON-stat
### eksempel HS-koder utenrikshandel

### Importerer biblioteker
Bruker [pyjstat](https://pypi.org/project/pyjstat/) og pandas. Pandas lastes som del av pyjstat

In [1]:
from pyjstat import pyjstat
import requests

### URL med tabellens metadata, som vi skal poste spørringen mot

In [2]:
tabid = "08799" # kam endres til årstabellen 08801, uten andre endringer
lang = "no" # language code can also be "en"
POST_URL = "https://data.ssb.no/api/v0/" + lang + "/table/" + tabid  # 'https://data.ssb.no/api/v0/no/table/08799'

Spørring, kan tas fra konsoll: Import / eksport alle varekoder til US siste måned, ca. 65000 celler. Max grense for en spørring i PxWebApi er 800.000 celler, inkl tomme celler

In [3]:
json_q = {
  "query": [
    {
      "code": "Varekoder",
      "selection": {
        "filter": "all",
        "values": [
          "*"
        ]
      }
    },
    {
      "code": "ImpEks",
      "selection": {
        "filter": "item",
        "values": [
          "1",
          "2"
        ]
      }
    },
    {
      "code": "Land",
      "selection": {
        "filter": "item",
        "values": [
          "US"
        ]
      }
    },
    {
      "code": "ContentsCode",
      "selection": {
        "filter": "item",
        "values": [
          "Mengde1",
          "Verdi",
          "Mengde2"
        ]
      }
    },
    {
      "code": "Tid",
      "selection": {
        "filter": "top",
        "values": [
          "1"
        ]
      }
    }
  ],
  "response": {
    "format": "json-stat2"
  }
}


### Poster spørringen

In [4]:
res = requests.post(POST_URL, json=json_q)

### Leser JSON-stat resultatet med biblioteket pyjstat
Lagrer det som datasett ds.

In [5]:
ds = pyjstat.Dataset.read(res.text)

In [6]:
type(ds)

pyjstat.pyjstat.Dataset

Se datasettet ds

In [7]:
# ds

### Henter noen overordnete metadata fra JSON-stat datasettet

In [8]:
tittel = ds['label']
print(tittel)

08799: Utenrikshandel med varer, etter varenummer, import/eksport, land, statistikkvariabel og måned


Sist Oppdatert som GMT

In [9]:
last_update = ds['updated']
print(last_update)

2022-11-15T07:00:00Z


Henter kilde

In [10]:
source = ds['source']
print(source)

Statistisk sentralbyrå


Roles gir noen snarveier til IDene på faste dimensjoner i datasettet

In [11]:
ds_roles = ds['role']
print(ds_roles)

OrderedDict([('time', ['Tid']), ('metric', ['ContentsCode']), ('geo', ['Land'])])


### Lager dataframes
Vi må lage to dataframes, en med tekst og en med ID. Pyjstat tillater bare enten/eller

In [12]:
hstrade = ds.write('dataframe')
hstrade_id = ds.write('dataframe', naming='id')

In [13]:
hstrade.head()

,varenummer,import/eksport,land,statistikkvariabel,måned,value
0,"Uspesifiserte varer, tollbare (1988-) (M1=kg, ...",Import,USA,Mengde 1 (M1),2022M10,0
1,"Uspesifiserte varer, tollbare (1988-) (M1=kg, ...",Import,USA,Verdi (kr),2022M10,0
2,"Uspesifiserte varer, tollbare (1988-) (M1=kg, ...",Import,USA,Mengde 2 (M2),2022M10,0
3,"Uspesifiserte varer, tollbare (1988-) (M1=kg, ...",Eksport,USA,Mengde 1 (M1),2022M10,0
4,"Uspesifiserte varer, tollbare (1988-) (M1=kg, ...",Eksport,USA,Verdi (kr),2022M10,0


In [14]:
hstrade_id.head()

,Varekoder,ImpEks,Land,ContentsCode,Tid,value
0,00000011_1988,1,US,Mengde1,2022M10,0
1,00000011_1988,1,US,Verdi,2022M10,0
2,00000011_1988,1,US,Mengde2,2022M10,0
3,00000011_1988,2,US,Mengde1,2022M10,0
4,00000011_1988,2,US,Verdi,2022M10,0


### Lager en ny kolonne med både HS varekode og tekst slått sammen

In [15]:
hstrade['hstrade_combi'] = hstrade_id['Varekoder'] + ' ' + hstrade['varenummer']

In [16]:
hstrade.columns

Index(['varenummer', 'import/eksport', 'land', 'statistikkvariabel', 'måned',
       'value', 'hstrade_combi'],
      dtype='object')

### Lager en ny dataframe som bare har med kolonnene vi ønsker, i ny rekkefølge. 
Merk doble [[ ]] for å lage dataframe

In [17]:
hstrade_new = hstrade[['hstrade_combi', 'import/eksport', 'land', 'statistikkvariabel', 'måned',
       'value']]

In [18]:
hstrade_new.tail(10)

,hstrade_combi,import/eksport,land,statistikkvariabel,måned,value
79544,99319900_2007 Utenlandsk proviant/forbruksart ...,Import,USA,Mengde 2 (M2),2022M10,0
79545,99319900_2007 Utenlandsk proviant/forbruksart ...,Eksport,USA,Mengde 1 (M1),2022M10,0
79546,99319900_2007 Utenlandsk proviant/forbruksart ...,Eksport,USA,Verdi (kr),2022M10,0
79547,99319900_2007 Utenlandsk proviant/forbruksart ...,Eksport,USA,Mengde 2 (M2),2022M10,0
79548,99999999_1988 Sum alle varenumre med restriksj...,Import,USA,Mengde 1 (M1),2022M10,15919265
79549,99999999_1988 Sum alle varenumre med restriksj...,Import,USA,Verdi (kr),2022M10,97291716
79550,99999999_1988 Sum alle varenumre med restriksj...,Import,USA,Mengde 2 (M2),2022M10,0
79551,99999999_1988 Sum alle varenumre med restriksj...,Eksport,USA,Mengde 1 (M1),2022M10,54329558
79552,99999999_1988 Sum alle varenumre med restriksj...,Eksport,USA,Verdi (kr),2022M10,441600731
79553,99999999_1988 Sum alle varenumre med restriksj...,Eksport,USA,Mengde 2 (M2),2022M10,0
